In [491]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [492]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/home/jovyan/work/running/素材" , static_folder = "/home/jovyan/work/running/素材")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [493]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [494]:
'''

設定訂閱頻道後顯示的圖文
製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
    TextSendMessage(text="您好! 歡迎使用我們的頻道"),
    TextSendMessage(text="RunningAI可以協助您找出適合的運動路線、並讓您在運動時有更好的體驗!"),
    ImageSendMessage(original_content_url='%s/home/jovyan/work/running/素材/girl.jpg' %server_url,
    preview_image_url='%s/home/jovyan/work/running/素材/girl.jpg' %server_url)
]


In [495]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/png','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [496]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    dict1 = {'活動賽事': '2-1', '路線推薦': '1-1'}
    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]
    print(event.message.text)
    print(dict1.get(str(event.message.text)))
    replyJsonPath = "./素材/"+dict1.get(event.message.text)+".json"
    print(replyJsonPath)
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
        
    )
    print('process_text_message done')

In [497]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    
    
    query_string_dict = parse_qs(event.postback.data)  # 將字串結果依照特定符號條件轉換成字典
#     print(event.postback.data)
#     print('query_string_dict',query_string_dict)
    if 'type' in query_string_dict:
        dict1 = {'walk': '2-2-walk', 'run': '2-2-run', 'ride': '2-2-ride'}
        result_message_array =[]
        print(query_string_dict.get('type'))
        replyJsonPath = './素材/'+dict1.get(query_string_dict.get('type')[0])+".json"
        print(replyJsonPath)
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        print('process_postback_event done')
    elif 'type_locate' in query_string_dict:
    
        result_message_array =[]
        print(query_string_dict.get('type_locate'))
        replyJsonPath = './素材/2-3.json'
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
#         print(result_message_array)
#         result_message_array[0]['template']['columns'][1]['actions'][1]['text'] = '騙你的'
#         result_message_array[0].template.columns[1].actions[1].label = '還是騙你的'
#         print(result_message_array[0].template.columns[1].actions[1].text)
#         print(result_message_array)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        print('process_postback_event done')
    elif 'sport' in query_string_dict:
        dict1 = {'climb': '1-2-climb', 'run': '1-2-run'}
        result_message_array =[]
        print(query_string_dict.get('sport'))
        replyJsonPath = './素材/'+dict1.get(query_string_dict.get('sport')[0])+".json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
#         print(result_message_array)
#         result_message_array[0]['template']['columns'][1]['actions'][1]['text'] = '騙你的'
#         result_message_array[0].template.columns[1].actions[1].label = '還是騙你的'
#         print(result_message_array[0].template.columns[1].actions[1].text)
#         print(result_message_array)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array 
        )
        print('process_postback_event done')
    elif 'sport_locate' in query_string_dict:
        dict1 = {'run:PD': '1-3-run-PD', 'run:KS': '1-3-run-KS','run:TC': '1-3-run-TC', 
                 'climb:PD': '1-3-climb-PD', 'climb:KS': '1-3-climb-KS','climb:TC': '1-3-climb-TC', }
        result_message_array =[]
        print(query_string_dict.get('sport_locate'))
        replyJsonPath = './素材/'+dict1.get(query_string_dict.get('sport_locate')[0])+".json"
        print(replyJsonPath)
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
#         print(result_message_array)
#         result_message_array[0]['template']['columns'][1]['actions'][1]['text'] = '騙你的'
#         result_message_array[0].template.columns[1].actions[1].label = '還是騙你的'
#         print(result_message_array[0].template.columns[1].actions[1].text)
#         print(result_message_array)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        print('process_postback_event done')


In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
[2019-10-25 07:40:56,872] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-492-f7a10dff5a44>", line 46, in 

['climb:PD']
./素材/1-3-climb-PD.json
['climb:PD']
./素材/1-3-climb-PD.json


172.20.0.2 - - [25/Oct/2019 07:41:53] "POST / HTTP/1.1" 200 -


process_postback_event done


172.20.0.2 - - [25/Oct/2019 07:41:59] "POST / HTTP/1.1" 200 -
[2019-10-25 07:42:00,833] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-492-f7a10dff5a44>", line 46,

強
None


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])